In [1]:
%pip install transformers torch pandas scikit-learn matplotlib
%pip install accelerate bitsandbytes

In [ ]:
#Importations et configuration pour Mistral
import pandas as pd
import torch
from transformers import pipeline, set_seed, AutoTokenizer, AutoModelForCausalLM

# --- Configuration du modèle Mistral 7B Instruct ---
MODELE_NOM = "mistralai/Mistral-7B-Instruct-v0.2"

print(f"Chargement du modèle : {MODELE_NOM}...")

# Chargement du Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODELE_NOM)

# Chargement du Modèle avec optimisation 4-bit (pour économiser la VRAM du GPU)
# Necessite les librairies accelerate et bitsandbytes
model = AutoModelForCausalLM.from_pretrained(
	MODELE_NOM,
	load_in_4bit=True,
	dtype=torch.float16, # Utiliser float16 pour optimiser l'utilisation du GPU
	device_map="auto" # Laisse Hugging Face décider où placer les couches (généralement sur le GPU)
)

# Création du Pipeline de génération
generator = pipeline(
	'text-generation', 
	model=model, 
	tokenizer=tokenizer
)

set_seed(42) # Pour rendre les résultats reproductibles
print("Modèle Mistral-7B-Instruct chargé en 4-bit.")

Chargement du modèle : mistralai/Mistral-7B-Instruct-v0.2...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


Modèle Mistral-7B-Instruct chargé en 4-bit.


In [ ]:
# Cellule 3 : Création du dataset Harry Potter (Mis à Jour)
import pandas as pd

hp_questions_data = [
	{
		"id": 1,
		"question": "Quelle est la tâche de la première année que Harry Potter réussit, impliquant un sortilège de lévitation ?",
		"contexte": "Hermione Granger est la première à maîtriser le sortilège de lévitation, Wingardium Leviosa, bien avant Ron Weasley. Ce sort permet de faire voler des objets.",
		"reponse_reference": "Le premier obstacle que Harry doit surmonter en première année est de récupérer la Clé Volante (une clé enchantée) en utilisant son talent de vol sur un balai."
	},
	{
		"id": 2,
		"question": "Expliquez brièvement le rôle du Retourneur de Temps et pourquoi il a été utilisé dans Le Prisonnier d'Azkaban.",
		"contexte": "Le Retourneur de Temps est un petit sablier porté autour du cou, permettant à son utilisateur de revivre des heures spécifiques du passé, sous certaines conditions strictes.",
		"reponse_reference": "Le Retourneur de Temps est un dispositif magique qui permet de voyager dans le temps. Il a été utilisé par Hermione Granger pour suivre plus de cours que possible, puis par Harry et Hermione pour sauver Buckbeak (l'hippogriffe) de l'exécution et pour libérer Sirius Black de prison."
	},
	{
		"id": 3,
		"question": "Quels sont les quatre objets que Tom Jedusor a transformés en Horcruxes avant le Serpent de Salazar Serpentard ?",
		"contexte": "Un Horcrux est un objet dans lequel un sorcier a dissimulé une partie de son âme, se rendant ainsi immortel tant qu'il n'est pas détruit.",
		"reponse_reference": "Les quatre premiers objets transformés en Horcruxes par Voldemort sont : le Journal intime de Tom Jedusor, la Bague de Poudlard de son grand-père (Marmorus Gaunt), le Médaillon de Salazar Serpentard et la Coupe de Helga Poufsouffle."
	},
	{
		"id": 4,
		"question": "Outre le Nimbus 2000, citez un balai magique notable utilisé par Harry Potter et expliquez son avantage.",
		"contexte": "La technologie des balais magiques évolue constamment. À l'origine, le Nimbus était l'étalon de la performance, mais il a été surpassé par des modèles plus récents.",
		"reponse_reference": "Harry a ensuite utilisé l'Éclair de Feu (Firebolt), qui est considéré comme le balai de course le plus rapide et le plus performant du monde magique. Il a été offert anonymement par Sirius Black après la destruction de son Nimbus 2000."
	},
	{
		"id": 5,
		"question": "Décrivez brièvement la relation entre le professeur Severus Rogue et Lily Evans (la mère d'Harry).",
		"contexte": "Rogue a été un Mangemort avant de se ranger du côté de Dumbledore. Son Patronus, une biche, est identique à celui de Lily.",
		"reponse_reference": "Rogue et Lily étaient amis d'enfance. L'amour non réciproque de Rogue pour Lily Evans est le moteur de son changement d'allégeance et de sa protection constante envers Harry. Son amour était 'Toujours' (Always)."
	},
	{
		"id": 6,
		"question": "Quel est le rôle principal de l'Ordre du Phénix et qui en était le chef à l'origine ?",
		"contexte": "L'Ordre est une société secrète de sorciers et sorcières fondée par Albus Dumbledore pour combattre Lord Voldemort et ses partisans (les Mangemorts).",
		"reponse_reference": "L'Ordre du Phénix est une organisation secrète dédiée à la lutte contre Lord Voldemort et à la défense du monde sorcier. Son chef incontesté est le professeur Albus Dumbledore, qui l'a reformé après le retour de Voldemort."
	},
	{
		"id": 7,
		"question": "Quel est le nom du sortilège impardonnable utilisé pour torturer la victime, et qui l'a souvent subi parmi les personnages principaux ?",
		"contexte": "Il existe trois Sortilèges Impardonnables : l'Imperium, le Doloris et l'Avada Kedavra. Utiliser ces sorts contre un humain est puni par une peine à vie à Azkaban.",
		"reponse_reference": "Le sortilège est le Sortilège Doloris (Crucio), qui provoque une douleur atroce. Les parents de Neville Londubat, Franck et Alice Londubat, ont été torturés par Bellatrix Lestrange avec ce sortilège jusqu'à la folie."
	},
	{
		"id": 8,
		"question": "Qu'est-ce que le Miroir du Riséd et quelle est l'avertissement essentiel que Dumbledore donne à son sujet ?",
		"contexte": "Le miroir est découvert par Harry dans L'École des Sorciers. Il est gardé par Dumbledore.",
		"reponse_reference": "Le Miroir du Riséd (Erised, en verlan) montre à la personne qui se regarde le désir le plus profond de son cœur. L'avertissement de Dumbledore est : 'Les hommes ont dépéri devant ce miroir, fascinés par ce qu'ils voyaient. Ne rêver que de ces choses-là, c'est oublier de vivre.'"
	},
	{
		"id": 9,
		"question": "Citez et décrivez brièvement le rôle de l'animal de compagnie de Hagrid qui est une araignée géante.",
		"contexte": "Hagrid a toujours eu un faible pour les créatures magiques dangereuses, même celles qui sont interdites, comme les acromantules.",
		"reponse_reference": "L'animal est Aragog, une Acromantule (araignée géante et douée de parole). Il a été élevé en secret par Hagrid. Aragog et sa colonie sont très dangereux mais sont restés fidèles à Hagrid."
	},
	{
		"id": 10,
		"question": "Quelle est la substance magique essentielle qui protège la Pierre Philosophale et comment est-elle créée ?",
		"contexte": "La Pierre Philosophale est capable de produire l'élixir de Vie, qui permet de vivre éternellement, et de transformer les métaux en or pur.",
		"reponse_reference": "Cette substance est l'élixir de Vie, un liquide rouge vif. Elle est créée par la Pierre Philosophale, fabriquée par l'alchimiste Nicolas Flamel, le seul à avoir réussi cet exploit."
	},
	{
		"id": 11,
		"question": "Qui est le fondateur de la maison Poufsouffle et quelle était sa philosophie pour accepter les élèves ?",
		"contexte": "Les quatre fondateurs de Poudlard étaient Godric Gryffondor, Helga Poufsouffle, Salazar Serpentard et Rowena Serdaigle.",
		"reponse_reference": "La fondatrice de la maison Poufsouffle est Helga Poufsouffle. Sa philosophie était la plus ouverte : elle acceptait tous les élèves qui n'entraient pas dans les critères des autres maisons, valorisant la loyauté, la patience, le travail acharné et l'équité."
	},
	{
		"id": 12,
		"question": "Quel est le sortilège utilisé pour générer un Patronus, et quelle condition émotionnelle est indispensable à sa réussite ?",
		"contexte": "Le Patronus est un charme très complexe, souvent considéré comme un marqueur de grande puissance magique.",
		"reponse_reference": "Le sortilège est Expecto Patronum. La condition indispensable est de se concentrer sur un souvenir extrêmement puissant et heureux. Cette concentration de joie permet de matérialiser l'essence du Patronus."
	},
	{
		"id": 13,
		"question": "Expliquez la signification du statut de sang de Né-Moldu (Muggle-born) dans le monde sorcier.",
		"contexte": "Ce terme est souvent utilisé de manière péjorative par les sorciers de sang-pur comme Drago Malefoy, qui les appellent « Sang-de-Bourbe ».",
		"reponse_reference": "Un Né-Moldu est un sorcier ou une sorcière né(e) de parents non magiques (Moldu). Cela signifie que la magie apparaît de manière inattendue dans la famille. Ce statut n'a aucune incidence sur la puissance magique de l'individu (ex: Hermione Granger)."
	},
	{
		"id": 14,
		"question": "Quel est le rôle de la Pensine (Penseive) et quel est l'avantage de l'utiliser ?",
		"contexte": "Dumbledore possède une Pensine. Rogue s'en sert également pour se débarrasser de ses pensées.",
		"reponse_reference": "La Pensine est un bassin en pierre enchanté utilisé pour extraire, stocker et visionner les souvenirs. L'avantage est qu'elle permet à l'utilisateur de visualiser les souvenirs en tant que spectateur extérieur, facilitant l'analyse des émotions et des événements."
	},
	{
		"id": 15,
		"question": "Qui sont les Maraudeurs, et quel est l'objet magique qu'ils ont créé ensemble ?",
		"contexte": "Les Maraudeurs ont été les meilleurs amis à Poudlard. Leurs surnoms sont liés à leurs formes Animagus.",
		"reponse_reference": "Les Maraudeurs sont quatre anciens amis de Poudlard : James Potter (Cornedrue), Sirius Black (Patmol), Remus Lupin (Lunard) et Peter Pettigrow (Queudver). Ils ont créé ensemble la Carte du Maraudeur, un document qui révèle l'emplacement de toutes les personnes se trouvant à Poudlard."
	}
]

df_hp = pd.DataFrame(hp_questions_data)
print(f"Dataset créé avec {len(df_hp)} questions. Prêt pour l'expérimentation.")
print(df_hp[['id', 'question', 'reponse_reference']].head())

Dataset créé avec 15 questions. Prêt pour l'expérimentation.
   id                                           question  \
0   1  Quelle est la tâche de la première année que H...   
1   2  Expliquez brièvement le rôle du Retourneur de ...   
2   3  Quels sont les quatre objets que Tom Jedusor a...   
3   4  Outre le Nimbus 2000, citez un balai magique n...   
4   5  Décrivez brièvement la relation entre le profe...   

                                   reponse_reference  
0  Le premier obstacle que Harry doit surmonter e...  
1  Le Retourneur de Temps est un dispositif magiq...  
2  Les quatre premiers objets transformés en Horc...  
3  Harry a ensuite utilisé l'Éclair de Feu (Fireb...  
4  Rogue et Lily étaient amis d'enfance. L'amour ...  


In [ ]:
#Fonctions pour construire les Prompts

SYSTEM_INSTRUCTION = "Tu es un assistant expert sur l'univers d'Harry Potter. Réponds de manière factuelle et concise."

def formater_prompt_mistral(prompt_utilisateur: str) -> str:
	"""Applique le format d'instruction strict de Mistral."""
	# Le format est : <s>[INST] Instruction [/INST]
	return f"<s>[INST] {prompt_utilisateur} [/INST]"

In [ ]:
def creer_prompt_zero_shot(question_actuelle: str) -> str:
	"""Crée un prompt Zero-Shot."""
	
	prompt_utilisateur = (
		f"{SYSTEM_INSTRUCTION}\n\n"
		f"Question: {question_actuelle}\n"
		f"Réponse:"
	)
	
	return formater_prompt_mistral(prompt_utilisateur)

In [ ]:
def creer_prompt_one_shot(question_actuelle: str, df_data: pd.DataFrame) -> str:
	"""Crée un prompt One-Shot en utilisant la première entrée du DataFrame comme exemple."""
	
	# Récupérer le premier exemple
	exemple = df_data.iloc[0]
	
	exemples_prompt = (
		f"Question: {exemple['question']}\n"
		f"Réponse: {exemple['reponse_reference']}\n\n"
	)
	
	prompt_utilisateur = (
		f"{SYSTEM_INSTRUCTION}\n\n"
		f"--- Exemple de Format ---\n"
		f"{exemples_prompt}"
		f"--- Fin de l'Exemple ---\n\n"
		f"Question: {question_actuelle}\n"
		f"Réponse:"
	)
	
	return formater_prompt_mistral(prompt_utilisateur)

In [ ]:
def creer_prompt_few_shot(question_actuelle: str, df_data: pd.DataFrame, nb_exemples: int = 3) -> str:
	"""Crée un prompt Few-Shot en utilisant les N premières entrées du DataFrame comme exemples."""
	
	# Récupérer les N premiers exemples (s'assurer qu'on ne dépasse pas la taille du DF)
	exemples = df_data.head(nb_exemples)
	
	exemples_prompt = ""
	for index, row in exemples.iterrows():
		# Utiliser le format simple Q:X, R:Y
		exemples_prompt += (
			f"Question: {row['question']}\n"
			f"Réponse: {row['reponse_reference']}\n\n"
		)
		
	prompt_utilisateur = (
		f"{SYSTEM_INSTRUCTION}\n\n"
		f"--- Exemples de Format ({nb_exemples} au total) ---\n"
		f"{exemples_prompt}"
		f"--- Fin des Exemples ---\n\n"
		f"Question: {question_actuelle}\n"
		f"Réponse:"
	)
	
	return formater_prompt_mistral(prompt_utilisateur)

In [15]:
#Génération de Réponse

import re # Assurez-vous que 're' est importé

def obtenir_reponse_llm(prompt_final: str) -> str:
	"""
	Appelle le pipeline de génération (Mistral 7B) avec des paramètres de base 
	(T=0.1 pour la précision) et nettoie la réponse.
	"""
	
	# Paramètres de base pour privilégier la précision factuelle (basse température)
	base_args = {
		"temperature": 0.1,  
		"top_p": 0.9,
		"top_k": 50,
		"do_sample": True, # Même à 0.1, on utilise le sampling
		"num_return_sequences": 1,
		"return_full_text": False
	}

	try:
		# Génération du texte via le pipeline 'generator'
		# Assurez-vous que la variable 'generator' a été définie dans la Cellule 2
		resultat = generator(
			prompt_final,
			**base_args
		)
		
		reponse_brute = resultat[0]['generated_text']
		
		# Nettoyage : retirer les balises et le prompt initial de Mistral
		reponse_llm = reponse_brute.strip()
		reponse_llm = re.sub(r'</s>', '', reponse_llm, flags=re.DOTALL).strip()
		
		# On cherche le dernier [/INST] pour séparer l'instruction de la réponse
		if "[/INST]" in reponse_llm:
			reponse_llm = reponse_llm.split("[/INST]")[-1].strip()
		
		return reponse_llm

	except Exception as e:
		# En cas d'erreur de génération (souvent lié au GPU)
		print(f"Erreur de génération LLM: {e}")
		return "ERREUR_GENERATION_LLM"

# --- Test rapide ---
question_test = "Comment est la famille d'Harry Potter ?"
prompt_test = creer_prompt_few_shot(question_test, df_hp, 10)
print(f"Prompt (début): {prompt_test}")

reponse_simple = obtenir_reponse_llm(prompt_test)
print(f"\nRéponse LLM: {reponse_simple}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt (début): <s>[INST] Tu es un assistant expert sur l'univers d'Harry Potter. Réponds de manière factuelle et concise.

--- Exemples de Format (10 au total) ---
Question: Quelle est la tâche de la première année que Harry Potter réussit, impliquant un sortilège de lévitation ?
Réponse: Le premier obstacle que Harry doit surmonter en première année est de récupérer la Clé Volante (une clé enchantée) en utilisant son talent de vol sur un balai.

Question: Expliquez brièvement le rôle du Retourneur de Temps et pourquoi il a été utilisé dans Le Prisonnier d'Azkaban.
Réponse: Le Retourneur de Temps est un dispositif magique qui permet de voyager dans le temps. Il a été utilisé par Hermione Granger pour suivre plus de cours que possible, puis par Harry et Hermione pour sauver Buckbeak (l'hippogriffe) de l'exécution et pour libérer Sirius Black de prison.

Question: Quels sont les quatre objets que Tom Jedusor a transformés en Horcruxes avant le Serpent de Salazar Serpentard ?
Réponse: Le